In [37]:
from altair import Chart, Color, Theta, Tooltip
from pandas import DataFrame

from app.data import MongoDB


In [38]:
DB = MongoDB()


In [39]:
mentors = DB.read("Mentors", {
    "validate_status": "approved",
    "is_active": True,
    "accepting_new_mentees": True,
})


In [40]:
df = DataFrame(mentors)[["tech_stack"]]
df


,tech_stack
0,"[iOS, Data Science]"
1,[Design UI/UX]
2,"[Design UI/UX, Career Development, Frontend]"
3,"[Design UI/UX, Career Development]"
4,"[Design UI/UX, iOS, Data Science]"
5,"[Data Science, Android, Frontend]"
6,"[Career Development, iOS, Data Science]"
7,"[Design UI/UX, iOS, Backend]"


In [41]:
type(df)

pandas.core.frame.DataFrame

In [43]:
df = df.explode(ignore_index=True, column="tech_stack")
df

,tech_stack
0,iOS
1,Data Science
2,Design UI/UX
3,Design UI/UX
4,Career Development
5,Frontend
6,Design UI/UX
7,Career Development
8,Design UI/UX
9,iOS


In [44]:
type(df)

pandas.core.frame.DataFrame

In [65]:
column = "tech_stack"
chart = Chart(
    df,
    title="Mentor Tech Stack"
).mark_arc(innerRadius=80).encode(
    color=Color(column),
    theta="count()",
    tooltip=Tooltip([column, "count()"]),
).properties(
    width=480,
    height=400,
    padding=24,
).configure(
    legend={"padding": 24},
    title={"fontSize": 20, "offset": 24},
    view={"stroke": "#FFF"},
)
chart

alt.Chart(...)